# Google Drive Interactions

> Fill in a module description here

In [1]:
#| default_exp google.drive

In [2]:
#| exporti

from fastcore.basics import patch_to
from nbdev.showdoc import *

import os
from enum import Enum
from dataclasses import dataclass ,field

import re
import datetime as dt
from dateutil.parser import parse as dtu_parse

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build, Resource


# from googleapiclient.errors import HttpError
# from google.auth.transport.requests import Request


import pptx2md
import zipfile
import io
import markdownify
from markdownify import MarkdownConverter

from bs4 import BeautifulSoup
from PIL import Image


In [3]:
#| export

class GoogleDrive_MimeType_Enum(Enum):
    """for translating Google Drive types into 'exports as'"""
    
    docx  = 'application/vnd.openxmlformats-officedocument.wordprocessingml.document'
    pptx = 'application/vnd.openxmlformats-officedocument.presentationml.presentation'
    
    odt  = 'application/vnd.oasis.opendocument.text'
    rtf = 'application/rtf'
    pdf = 'application/pdf'
    txt = 'text/plain'
    # html = 'application/zip'
    zip = 'application/zip'
    epub  = 'application/epub+zip'
    
    folder = 'application/vnd.google-apps.folder'
    gdoc = 'application/vnd.google-apps.document'   
    gsheet ='application/vnd.google-apps.spreadsheet'
    gshortcut = 'application/vnd.google-apps.shortcut'
    gslides = 'application/vnd.google-apps.presentation'
    
    # 'tex'  = 'application/zip'
    # 'html.zip'= 'application/zip'

In [4]:
for member in GoogleDrive_MimeType_Enum:
    print(member.name, member.value)

docx application/vnd.openxmlformats-officedocument.wordprocessingml.document
pptx application/vnd.openxmlformats-officedocument.presentationml.presentation
odt application/vnd.oasis.opendocument.text
rtf application/rtf
pdf application/pdf
txt text/plain
zip application/zip
epub application/epub+zip
folder application/vnd.google-apps.folder
gdoc application/vnd.google-apps.document
gsheet application/vnd.google-apps.spreadsheet
gshortcut application/vnd.google-apps.shortcut
gslides application/vnd.google-apps.presentation


# Google Drive File Looper
retrieves a listing of all the files and subfiles in a folder

In [5]:
#| exporti
def get_date(datefield):
    return dtu_parse(datefield) if datefield else None


In [6]:
get_date('2021-01-01')

datetime.datetime(2021, 1, 1, 0, 0)

In [7]:
# | export

@dataclass
class GDoc_File:
    mime_type: str
    webViewLink: str
    doc_id: str
    doc_name: str
    modified_time: dt.datetime

    parent_ls : [str] = field(default = None)
    # parent_gdocs : list[dict] = field(default = None, repr = False)

    folder_path : str = None
    
    folder_path: str = None
    file_type: GoogleDrive_MimeType_Enum = None

    service: Resource = field(default = None, repr = False)
    creds: Credentials = field( default = None, repr = False)
    
    content = None

    def __post_init__(self):
        if self.creds and self.service is None:
            self.service = self._generate_service(creds = self.creds)

        # if self.parent_ls:
        #     print(self.parent_ls)
        #     self.parent_gdocs = [GDoc_File.get_from_id(document_id = parent, service = self.service) for parent in self.parent_ls]
        #     self.folder_path = " > ".join([ parent.doc_name for parent in self.parent_gdocs])

    
    @staticmethod
    def _generate_service(creds: Credentials ):
        return build(serviceName='drive',
                             version='v3', 
                             credentials=creds)

    @classmethod
    def _from_json(cls, obj: dict, service=None, creds=None):

        return cls(mime_type=obj['mimeType'],
                   webViewLink=obj['webViewLink'],
                   doc_id=obj['id'],
                   doc_name=obj['name'],
                   modified_time=get_date(obj['modifiedTime']),
                   parent_ls = obj.get('parents', None),
                   
                   service=service,
                   creds=creds)


In [8]:
#export
class GDoc_File_ServiceRequired(Exception):
    def __init__(self, doc_url = None, doc_id = None):
        message = f"service (googleclientapi.discovery.Resource) required to download {doc_url or doc_id}"
        super().__init__(message)
        

In [9]:
@patch_to(GDoc_File, cls_method=True)
def get_from_id(cls,
                document_id : str,
                creds: Credentials = None,
                service: Resource = None,
                return_raw: bool = False
                ):

    if not service and creds:
        service = cls._generate_service(creds = creds) 

    if not service:
        raise GDoc_File_ServiceRequired(doc_id=document_id)

    document_data = service.files().get(fileId=document_id, fields="id,webViewLink, name, mimeType,modifiedTime,parents ").execute()

    if return_raw:
        return document_data

    return cls._from_json(obj = document_data, creds = creds, service = service)


#### sample implementation of get_from_id

In [10]:
from dotenv import load_dotenv
import json
import os
import gdoc_sync.google.creds as gcred

# DOCUMENT_ID = '1j7XsbvFy0xUgGL6i-3LSChKvzSmTZSOyimEt6tQS-Kk'
DOCUMENT_ID = '1m48jciWr2iZqwnhN7rezM8_GposSDZhGwQhsIdRqLJg'


# generates Credentials object
creds = gcred.get_creds_from_env(credentials_env_key= 'GDOC_PUBLIC_CLIENT', token_env_key = 'GDOC_TOKEN',  scope = gcred.SCOPE, env_file = '../.env')

GDoc_File.get_from_id(document_id= DOCUMENT_ID, creds = creds, return_raw= False)

GDoc_File(mime_type='application/vnd.google-apps.document', webViewLink='https://docs.google.com/document/d/1m48jciWr2iZqwnhN7rezM8_GposSDZhGwQhsIdRqLJg/edit?usp=drivesdk', doc_id='1m48jciWr2iZqwnhN7rezM8_GposSDZhGwQhsIdRqLJg', doc_name='test_subfolder', modified_time=datetime.datetime(2023, 8, 17, 21, 1, 6, 479000, tzinfo=tzutc()), parent_ls=['1grGeTxDXjEoo8MkRsMzTg6v_Pdtuxi1T'], folder_path=None, file_type=None)

# Handle Downloading Documents

In [11]:
#| exporti
def upsert_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

def convert_to_snake(text_str):
    """converts 'snake_case_str' to 'snakeCaseStr'"""

    return text_str.replace(" ", "_").lower()


def clean_url_name(path_name):
    valid_chars = r"[^a-zA-Z0-9_]"

    return re.sub(valid_chars, "", path_name)


def generate_file_name(raw_name):
    return clean_url_name(convert_to_snake(raw_name))

## handle pptx

In [12]:
#| exporti

def download_pptx(content, output_folder):
    output_ppt_index = os.path.join(output_folder, 'index.pptx')

    with open(output_ppt_index, "wb+") as binary_file:
        # Write bytes to file
        binary_file.write(content)

    pptx2md.convert(output_ppt_index,
                    output=os.path.join(output_folder, 'index.md'),
                    image_dir=os.path.join(output_folder, 'images'))
    
    return True

## handle doc

Download Zip will export the document as a .html file, convert it to beautiful soup and then use `markdownify` to convert HTML to markdown

In [13]:
# | exporti
def clean_bs4(html):
    soup = BeautifulSoup(html, "html.parser")
    return soup.find('body')

#| export
class ImageBlockConverter(MarkdownConverter):
    """
    Create a custom MarkdownConverter that adds two newlines after an image
    """

    def convert_img(self, el, text, convert_as_inline):

        style_obj = {(obj.split(':')[0].strip()) : obj.split(':')[1].strip() for obj in el.get('style').split(';') if ':' in obj}
        
        file_path = os.path.join(os.path.dirname(self.options['file_path']), el['src'])

        image = Image.open(file_path)

        width = style_obj['width'].replace('px', '')
        width = int(float(width))

        height = style_obj['height'].replace('px', '')
        height = int(float(height))


        new_image = image.resize((width,height))
        new_image.save(file_path)
    
        return super().convert_img(el, text, convert_as_inline)

# Create shorthand method for conversion
def md(html, **options):
    return ImageBlockConverter(**options).convert(html)


def convert_file_in_place(file_path):
    with(open(file_path, encoding='utf-8')) as f:
        html = f.read()
    
    h = md(str(html), keep_inline_images_in= ['td', 'span'], file_path =file_path)

    md_path = file_path.replace(".html", ".md")

    with open(md_path, "w+", encoding="utf-8") as f:
        f.write(h)

    return md_path

def download_zip(content, output_folder):
    zip = zipfile.ZipFile(io.BytesIO(content), 'r')
    zip.extractall(output_folder)

    file_ls = os.listdir(output_folder)

    # rename the html file to index.html
    for file_name in file_ls:
        if file_name.endswith('.html'):
            output_index = os.path.join(output_folder, 'index.html')
            os.replace(os.path.join(output_folder, file_name), output_index)
            convert_file_in_place(os.path.join(output_folder, 'index.html'))
    
    return True

# Download Factory
exted by adding more download functions

In [14]:
#| exporti

download_factory = {
    'application/vnd.google-apps.presentation': {'download_fn': download_pptx,
                                                 'export_type': GoogleDrive_MimeType_Enum.pptx},
    'default': {'download_fn': download_zip,
                'export_type': GoogleDrive_MimeType_Enum.zip}
}


In [15]:
#| export    
class GDocFile_DownloadError(Exception):
    def __init__(self, doc_url):
        message = f"failure to download content for {doc_url}"
        super().__init__(message)


@patch_to(GDoc_File)
def download_file(self :GDoc_File,
                  service : Resource = None,
                  output_folder: str = "",
                  is_use_file_id : bool = False
                  ):

    service = service or self.service
    factory = download_factory.get(self.mime_type, download_factory['default'])

    loop = 0
    content = None

    while loop < 3 and content is None:
        try:
            loop += 1
            content = service.files().export(
                fileId=self.doc_id,
                mimeType=factory['export_type'].value).execute()

            self.content = content

        except HttpError as err:
            print(err)

        except Exception as e:
            print(e)

    if not content:
        raise GDocFile_DownloadError(self.webViewLink)
    
    # create output_folder
    clean_file_name = self.doc_id if is_use_file_id else generate_file_name(self.doc_name)
    output_folder_path = os.path.join(output_folder, clean_file_name)
    upsert_folder(output_folder_path)

    # download
    return factory['download_fn'](content, output_folder_path)

### sample implementation of download_file

In [16]:
from dotenv import load_dotenv
import json
import os
import gdoc_sync.google.creds as gcred

DOCUMENT_ID = '1j7XsbvFy0xUgGL6i-3LSChKvzSmTZSOyimEt6tQS-Kk'

# generates Credentials object
creds = gcred.get_creds_from_env(credentials_env_key= 'GDOC_PUBLIC_CLIENT', token_env_key = 'GDOC_TOKEN',  scope = gcred.SCOPE, env_file = '../.env')

test_file = GDoc_File.get_from_id(document_id= DOCUMENT_ID, creds = creds, return_raw= False)

test_file.download_file(output_folder= 'sample', is_use_file_id=  True)

True

# Export Folders

In [17]:
# | exporti

def process_file(file_obj, folder_path):
    """gets the file type using GoogleDrive Enum, useful for processing & downloading a file"""

    if folder_path:
        file_obj.update({"folder_path": folder_path})

    file_type = "undefined_enum"
    try:
        file_type = GoogleDrive_MimeType_Enum(file_obj["mimeType"]).name

    except Exception as e:
        print(e)

    finally:
        file_obj.update({"type": file_type})
        return file_obj


def get_parent(parent_id, service):
    return service.files().get(
        fileId = parent_id,
        fields="id,webViewLink, name, mimeType,modifiedTime,parents",
    ).execute()

def get_file(file_id, service):
    res = service.files().get(
        fileId = file_id,
        fields="id,webViewLink, name, mimeType,modifiedTime,parents",
    ).execute()

    res['folder_path'] = get_parent(res.get('parents')[0], service = service).get('name')

    return res


def get_files(folder_id, service):
    """retrieves files in a folder"""
    page_token = None
    file_ls = []

    while True:
        res = (
            service.files()
            .list(
                q=f"'{folder_id}' in parents",
                pageSize=10,
                fields="nextPageToken, files(id,webViewLink, name, mimeType,modifiedTime,parents )",
                pageToken=page_token,
            )
            .execute()
        )


        parent_ls = list(set([parent for file in res.get('files') for parent in file['parents']]))
        parent_dict = { parent: get_parent(parent, service).get('name') for parent in parent_ls}
        print(parent_dict)


        items = res.get("files", [])

        [item.update({'folder_path': parent_dict.get(item['parents'][0])}) for item in items]

        file_ls += items

        page_token = res.get("nextPageToken", None)

        if page_token is None:
            break
    
    return file_ls


def get_file_looper(folder_id, service, file_ls=None, folder_path= ""):
    if not file_ls:
        file_ls = []

    """recursive function to get files in a folder and map over files in subfolder"""

    new_files = get_files(folder_id=folder_id, service=service)

    if len(new_files) == 0 : 
        file_ls = [ get_file(file_id = folder_id, service = service)]


    new_files = [process_file(file_obj, folder_path) for file_obj in new_files]

    file_ls += new_files

    if new_files:
        for file_obj in new_files:
            if file_obj["type"] == "folder":
                sub_folder_path = os.path.join(
                    folder_path, generate_file_name(file_obj.get("name"))
                )

                get_file_looper(
                    folder_id=file_obj.get("id"),
                    folder_path=sub_folder_path,
                    service=service,
                    file_ls=file_ls,
                )
    
        
    return file_ls

In [18]:
#| export

def get_folder_contents(folder_id, creds, folder_path='root', return_raw: bool = False) -> [dict]: # returns list of file objects

    """creates a list files in a folder using a set of credentials"""

    service = build(serviceName='drive',
                    version='v3',
                    credentials=creds)

    return get_file_looper(folder_id=folder_id, service=service, folder_path=folder_path, )


In [19]:
import gdoc_sync.google.creds as gcred

import pandas as pd

FOLDER_ID ='1SRrD1dNgZgHYjnhkJtARbLhydMP94qWi'
DOCUMENT_ID = '1j7XsbvFy0xUgGL6i-3LSChKvzSmTZSOyimEt6tQS-Kk'
DOCUMENT_SUBFOLDER_ID = '1m48jciWr2iZqwnhN7rezM8_GposSDZhGwQhsIdRqLJg'

# generates Credentials object
creds = gcred.get_creds_from_env(credentials_env_key= 'GDOC_PUBLIC_CLIENT', token_env_key = 'GDOC_TOKEN',  scope = gcred.SCOPE, env_file = '../.env')


res = get_folder_contents(folder_id = DOCUMENT_ID, creds = creds, return_raw= False)

pd.DataFrame(res)

{}


,id,name,mimeType,parents,webViewLink,modifiedTime,folder_path
0,1j7XsbvFy0xUgGL6i-3LSChKvzSmTZSOyimEt6tQS-Kk,Sample Doc for Google_Sync project,application/vnd.google-apps.document,[1SRrD1dNgZgHYjnhkJtARbLhydMP94qWi],https://docs.google.com/document/d/1j7XsbvFy0x...,2023-11-27T19:39:00.131Z,gdoc_sync_sample


In [20]:
#| hide
import nbdev; 
nbdev.nbdev_export()